#### Notes:

Currently finished up to Evaluator.

#### Next Steps:

Finish through model serving

Integrate Kubeflow

##### Resources:

https://www.tensorflow.org/tfx/tutorials

https://www.tensorflow.org/tfx/tutorials/tfx/components_keras

In [1]:
try:
    import colab
    !pip install --upgrade pip
except:
    pass

In [2]:
!pip install -U tfx

In [3]:
import os
import pprint
import tempfile
import urllib

import absl
import tensorflow as tf
import tensorflow_model_analysis as tfma
tf.get_logger().propagate = False
pp = pprint.PrettyPrinter()

from tfx import v1 as tfx
from tfx.orchestration.experimental.interactive.interactive_context import InteractiveContext

%load_ext tfx.orchestration.experimental.interactive.notebook_extensions.skip

In [4]:
print('TensorFlow version: {}'.format(tf.__version__))
print('TFX version: {}'.format(tfx.__version__))

TensorFlow version: 2.5.0
TFX version: 1.0.0


In [5]:
# This is the root directory for your TFX pip package installation.
_tfx_root = tfx.__path__[0]

# This is the directory containing the TFX Loan Pipeline.
_loan_root = os.path.join(_tfx_root, 'examples/loan_data_pipeline')

# This is the path where your model will be pushed for serving.
_serving_model_dir = os.path.join(
    tempfile.mkdtemp(), 'serving_model/loan_simple')

# Set up logging.
absl.logging.set_verbosity(absl.logging.INFO)

In [6]:
_data_root = tempfile.mkdtemp(prefix='tfx-data')
DATA_PATH = 'https://raw.githubusercontent.com/Jeromeschmidt/LendingClubLoanData/main/data/1.csv'
_data_filepath = os.path.join(_data_root, "data.csv")
urllib.request.urlretrieve(DATA_PATH, _data_filepath)

('/tmp/tfx-datayhfttu5q/data.csv', <http.client.HTTPMessage at 0x7fd9d6575310>)

In [7]:
!head {_data_filepath}

id,member_id,loan_amnt,funded_amnt,funded_amnt_inv,term,int_rate,installment,grade,sub_grade,emp_title,emp_length,home_ownership,annual_inc,verification_status,issue_d,loan_status,pymnt_plan,url,desc,purpose,title,zip_code,addr_state,dti,delinq_2yrs,earliest_cr_line,fico_range_low,fico_range_high,inq_last_6mths,mths_since_last_delinq,mths_since_last_record,open_acc,pub_rec,revol_bal,revol_util,total_acc,initial_list_status,out_prncp,out_prncp_inv,total_pymnt,total_pymnt_inv,total_rec_prncp,total_rec_int,total_rec_late_fee,recoveries,collection_recovery_fee,last_pymnt_d,last_pymnt_amnt,next_pymnt_d,last_credit_pull_d,last_fico_range_high,last_fico_range_low,collections_12_mths_ex_med,mths_since_last_major_derog,policy_code,application_type,annual_inc_joint,dti_joint,verification_status_joint,acc_now_delinq,tot_coll_amt,tot_cur_bal,open_acc_6m,open_act_il,open_il_12m,open_il_24m,mths_since_rcnt_il,total_bal_il,il_util,open_rv_12m,open_rv_24m,max_bal_bc,all_util,total_rev_hi_lim,inq_fi,to

In [8]:
context = InteractiveContext()

In [9]:
example_gen = tfx.components.CsvExampleGen(input_base=_data_root)
context.run(example_gen)

INFO:absl:Running driver for CsvExampleGen
INFO:absl:MetadataStore with DB connection initialized
INFO:absl:select span and version = (0, None)
INFO:absl:latest span and version = (0, None)
INFO:absl:Running executor for CsvExampleGen
INFO:absl:Generating examples.


INFO:absl:Processing input csv data /tmp/tfx-datayhfttu5q/* to TFExample.
INFO:absl:Examples generated.
INFO:absl:Running publisher for CsvExampleGen
INFO:absl:MetadataStore with DB connection initialized


ExecutionResult(
    component_id: CsvExampleGen
    execution_id: 1
    outputs:
        examples: Channel(
            type_name: Examples
            artifacts: [Artifact(artifact: id: 1
        type_id: 5
        uri: "/tmp/tfx-interactive-2021-08-09T00_37_07.628346-35rc043i/CsvExampleGen/examples/1"
        properties {
          key: "split_names"
          value {
            string_value: "[\"train\", \"eval\"]"
          }
        }
        custom_properties {
          key: "input_fingerprint"
          value {
            string_value: "split:single_split,num_files:1,total_bytes:7482542,xor_checksum:1628469427,sum_checksum:1628469427"
          }
        }
        custom_properties {
          key: "payload_format"
          value {
            string_value: "FORMAT_TF_EXAMPLE"
          }
        }
        custom_properties {
          key: "span"
          value {
            int_value: 0
          }
        }
        custom_properties {
          key: "state"
          value {
            string_value: "published"
          }
        }
        custom_properties {
          key: "tfx_version"
          value {
            string_value: "1.0.0"
          }
        }
        state: LIVE
        , artifact_type: id: 5
        name: "Examples"
        properties {
          key: "span"
          value: INT
        }
        properties {
          key: "split_names"
          value: STRING
        }
        properties {
          key: "version"
          value: INT
        }
        )]
            additional_properties: {}
            additional_custom_properties: {}
        ))

In [10]:
artifact = example_gen.outputs['examples'].get()[0]
print(artifact.split_names, artifact.uri)

["train", "eval"] /tmp/tfx-interactive-2021-08-09T00_37_07.628346-35rc043i/CsvExampleGen/examples/1


In [11]:
# Get the URI of the output artifact representing the training examples, which is a directory
loan_uri = os.path.join(example_gen.outputs['examples'].get()[0].uri, 'Split-train')

# Get the list of files in this directory (all compressed TFRecord files)
tfrecord_filenames = [os.path.join(loan_uri, name)
                      for name in os.listdir(loan_uri)]

# Create a `TFRecordDataset` to read these files
dataset = tf.data.TFRecordDataset(tfrecord_filenames, compression_type="GZIP")

# # Iterate over the first 3 records and decode them.
# for tfrecord in dataset.take(3):
#   serialized_example = tfrecord.numpy()
#   example = tf.train.Example()
#   example.ParseFromString(serialized_example)
#   pp.pprint(example)

In [12]:
statistics_gen = tfx.components.StatisticsGen(
    examples=example_gen.outputs['examples'])
context.run(statistics_gen)

INFO:absl:Excluding no splits because exclude_splits is not set.
INFO:absl:Running driver for StatisticsGen
INFO:absl:MetadataStore with DB connection initialized
INFO:absl:Running executor for StatisticsGen
INFO:absl:Generating statistics for split train.
INFO:absl:Statistics for split train written to /tmp/tfx-interactive-2021-08-09T00_37_07.628346-35rc043i/StatisticsGen/statistics/2/Split-train.
INFO:absl:Generating statistics for split eval.
INFO:absl:Statistics for split eval written to /tmp/tfx-interactive-2021-08-09T00_37_07.628346-35rc043i/StatisticsGen/statistics/2/Split-eval.
INFO:absl:Running publisher for StatisticsGen
INFO:absl:MetadataStore with DB connection initialized


ExecutionResult(
    component_id: StatisticsGen
    execution_id: 2
    outputs:
        statistics: Channel(
            type_name: ExampleStatistics
            artifacts: [Artifact(artifact: id: 2
        type_id: 7
        uri: "/tmp/tfx-interactive-2021-08-09T00_37_07.628346-35rc043i/StatisticsGen/statistics/2"
        properties {
          key: "split_names"
          value {
            string_value: "[\"train\", \"eval\"]"
          }
        }
        custom_properties {
          key: "name"
          value {
            string_value: "statistics"
          }
        }
        custom_properties {
          key: "producer_component"
          value {
            string_value: "StatisticsGen"
          }
        }
        custom_properties {
          key: "state"
          value {
            string_value: "published"
          }
        }
        custom_properties {
          key: "tfx_version"
          value {
            string_value: "1.0.0"
          }
        }
        state: LIVE
        , artifact_type: id: 7
        name: "ExampleStatistics"
        properties {
          key: "span"
          value: INT
        }
        properties {
          key: "split_names"
          value: STRING
        }
        )]
            additional_properties: {}
            additional_custom_properties: {}
        ))

In [13]:
context.show(statistics_gen.outputs['statistics'])

In [14]:
schema_gen = tfx.components.SchemaGen(
    statistics=statistics_gen.outputs['statistics'],
    infer_feature_shape=False)
context.run(schema_gen)

INFO:absl:Excluding no splits because exclude_splits is not set.
INFO:absl:Running driver for SchemaGen
INFO:absl:MetadataStore with DB connection initialized
INFO:absl:Running executor for SchemaGen
INFO:absl:Processing schema from statistics for split train.
INFO:absl:Processing schema from statistics for split eval.
INFO:absl:Schema written to /tmp/tfx-interactive-2021-08-09T00_37_07.628346-35rc043i/SchemaGen/schema/3/schema.pbtxt.
INFO:absl:Running publisher for SchemaGen
INFO:absl:MetadataStore with DB connection initialized


ExecutionResult(
    component_id: SchemaGen
    execution_id: 3
    outputs:
        schema: Channel(
            type_name: Schema
            artifacts: [Artifact(artifact: id: 3
        type_id: 9
        uri: "/tmp/tfx-interactive-2021-08-09T00_37_07.628346-35rc043i/SchemaGen/schema/3"
        custom_properties {
          key: "name"
          value {
            string_value: "schema"
          }
        }
        custom_properties {
          key: "producer_component"
          value {
            string_value: "SchemaGen"
          }
        }
        custom_properties {
          key: "state"
          value {
            string_value: "published"
          }
        }
        custom_properties {
          key: "tfx_version"
          value {
            string_value: "1.0.0"
          }
        }
        state: LIVE
        , artifact_type: id: 9
        name: "Schema"
        )]
            additional_properties: {}
            additional_custom_properties: {}
        ))

In [15]:
context.show(schema_gen.outputs['schema'])

,Type,Presence,Valency,Domain
Feature name,,,,
'acc_now_delinq',FLOAT,required,single,-
'acc_open_past_24mths',FLOAT,required,single,-
'addr_state',STRING,required,single,'addr_state'
'all_util',FLOAT,required,"[0,1]",-
'annual_inc',FLOAT,required,single,-
...,...,...,...,...
'total_rev_hi_lim',FLOAT,required,single,-
'url',BYTES,required,single,-
'verification_status',STRING,required,single,'verification_status'


/opt/conda/lib/python3.7/site-packages/tensorflow_data_validation/utils/display_util.py:180: FutureWarning: Passing a negative integer is deprecated in version 1.0 and will not be supported in future version. Instead, use None to not limit the column width.
  pd.set_option('max_colwidth', -1)


,Values
Domain,
'addr_state',"'AK', 'AL', 'AR', 'AZ', 'CA', 'CO', 'CT', 'DC', 'DE', 'FL', 'GA', 'HI', 'IL', 'IN', 'KS', 'KY', 'LA', 'MA', 'MD', 'ME', 'MI', 'MN', 'MO', 'MS', 'MT', 'NC', 'ND', 'NE', 'NH', 'NJ', 'NM', 'NV', 'NY', 'OH', 'OK', 'OR', 'PA', 'RI', 'SC', 'SD', 'TN', 'TX', 'UT', 'VA', 'VT', 'WA', 'WI', 'WV', 'WY'"
'application_type',"'Individual', 'Joint App'"
'debt_settlement_flag',"'N', 'Y'"
'debt_settlement_flag_date',"'Apr-2018', 'Aug-2017', 'Aug-2018', 'Dec-2017', 'Dec-2018', 'Feb-2018', 'Feb-2019', 'Jan-2018', 'Jan-2019', 'Jul-2017', 'Jul-2018', 'Jun-2018', 'Mar-2018', 'Mar-2019', 'May-2017', 'May-2018', 'Nov-2017', 'Nov-2018', 'Oct-2017', 'Oct-2018', 'Sep-2017', 'Sep-2018', 'Apr-2017', 'Aug-2016', 'Jun-2017', 'Mar-2017'"
'desc',"'We knew that using our credit cards to finance an adoption would squeeze us, but then medical and other unexpected expenses made the situation almost impossible. We are a stable family in a stable community. We just need to break a cycle of debt that is getting worse.'"
'disbursement_method','Cash'
'emp_length',"'1 year', '10+ years', '2 years', '3 years', '4 years', '5 years', '6 years', '7 years', '8 years', '9 years', '< 1 year'"
'grade',"'A', 'B', 'C', 'D', 'E', 'F', 'G'"
'hardship_end_date',"'Apr-2018', 'Aug-2017', 'Dec-2017', 'Dec-2018', 'Feb-2018', 'Jan-2018', 'Jan-2019', 'Jul-2017', 'Jul-2018', 'Jun-2019', 'Mar-2018', 'May-2017', 'Nov-2017', 'Nov-2018', 'Oct-2017', 'Oct-2018', 'Sep-2017', 'May-2019', 'Sep-2018'"


In [16]:
example_validator = tfx.components.ExampleValidator(
    statistics=statistics_gen.outputs['statistics'],
    schema=schema_gen.outputs['schema'])
context.run(example_validator)

INFO:absl:Excluding no splits because exclude_splits is not set.
INFO:absl:Running driver for ExampleValidator
INFO:absl:MetadataStore with DB connection initialized
INFO:absl:Running executor for ExampleValidator
INFO:absl:Validating schema against the computed statistics for split train.
INFO:absl:Validation complete for split train. Anomalies written to /tmp/tfx-interactive-2021-08-09T00_37_07.628346-35rc043i/ExampleValidator/anomalies/4/Split-train.
INFO:absl:Validating schema against the computed statistics for split eval.
INFO:absl:Validation complete for split eval. Anomalies written to /tmp/tfx-interactive-2021-08-09T00_37_07.628346-35rc043i/ExampleValidator/anomalies/4/Split-eval.
INFO:absl:Running publisher for ExampleValidator
INFO:absl:MetadataStore with DB connection initialized


ExecutionResult(
    component_id: ExampleValidator
    execution_id: 4
    outputs:
        anomalies: Channel(
            type_name: ExampleAnomalies
            artifacts: [Artifact(artifact: id: 4
        type_id: 11
        uri: "/tmp/tfx-interactive-2021-08-09T00_37_07.628346-35rc043i/ExampleValidator/anomalies/4"
        properties {
          key: "split_names"
          value {
            string_value: "[\"train\", \"eval\"]"
          }
        }
        custom_properties {
          key: "name"
          value {
            string_value: "anomalies"
          }
        }
        custom_properties {
          key: "producer_component"
          value {
            string_value: "ExampleValidator"
          }
        }
        custom_properties {
          key: "state"
          value {
            string_value: "published"
          }
        }
        custom_properties {
          key: "tfx_version"
          value {
            string_value: "1.0.0"
          }
        }
        state: LIVE
        , artifact_type: id: 11
        name: "ExampleAnomalies"
        properties {
          key: "span"
          value: INT
        }
        properties {
          key: "split_names"
          value: STRING
        }
        )]
            additional_properties: {}
            additional_custom_properties: {}
        ))

In [17]:
context.show(example_validator.outputs['anomalies'])

/opt/conda/lib/python3.7/site-packages/tensorflow_data_validation/utils/display_util.py:217: FutureWarning: Passing a negative integer is deprecated in version 1.0 and will not be supported in future version. Instead, use None to not limit the column width.
  pd.set_option('max_colwidth', -1)


In [18]:
_loan_constants_module_file = 'loan_constants.py'

In [19]:
%%writefile {_loan_constants_module_file}

COLUMN_FLOAT = ['loan_amnt',
 'funded_amnt',
 'funded_amnt_inv',
 'int_rate',
 'installment',
 'annual_inc',
 'dti',
 'delinq_2yrs',
 'fico_range_low',
 'fico_range_high',
 'inq_last_6mths',
 'open_acc',
 'pub_rec',
 'revol_bal',
 'revol_util',
 'total_acc',
 'out_prncp',
 'out_prncp_inv',
 'total_pymnt',
 'total_pymnt_inv',
 'total_rec_prncp',
 'total_rec_int',
 'total_rec_late_fee',
 'recoveries',
 'collection_recovery_fee',
 'last_pymnt_amnt',
 'last_fico_range_high',
 'last_fico_range_low',
 'collections_12_mths_ex_med',
 'policy_code',
 'acc_now_delinq',
 'tot_coll_amt',
 'tot_cur_bal',
 'total_rev_hi_lim',
 'acc_open_past_24mths',
 'avg_cur_bal',
 'bc_open_to_buy',
 'bc_util',
 'chargeoff_within_12_mths',
 'delinq_amnt',
 'mo_sin_old_il_acct',
 'mo_sin_old_rev_tl_op',
 'mo_sin_rcnt_rev_tl_op',
 'mo_sin_rcnt_tl',
 'mort_acc',
 'mths_since_recent_bc',
 'mths_since_recent_inq',
 'num_accts_ever_120_pd',
 'num_actv_bc_tl',
 'num_actv_rev_tl',
 'num_bc_sats',
 'num_bc_tl',
 'num_il_tl',
 'num_op_rev_tl',
 'num_rev_accts',
 'num_rev_tl_bal_gt_0',
 'num_sats',
 'num_tl_120dpd_2m',
 'num_tl_30dpd',
 'num_tl_90g_dpd_24m',
 'num_tl_op_past_12m',
 'pct_tl_nvr_dlq',
 'percent_bc_gt_75',
 'pub_rec_bankruptcies',
 'tax_liens',
 'tot_hi_cred_lim',
 'total_bal_ex_mort',
 'total_bc_limit',
 'total_il_high_credit_limit']

COLUMN_OBJECT = ['term',
 'emp_length',
 'home_ownership',
 'verification_status',
 'issue_d',
 'loan_status',
 'pymnt_plan',
 'purpose',
 'zip_code',
 'addr_state',
 'earliest_cr_line',
 'initial_list_status',
 'last_pymnt_d',
 'last_credit_pull_d',
 'application_type',
 'hardship_flag',
 'disbursement_method',
 'debt_settlement_flag']

LABEL_KEY = 'grade'

BUCKET_SIZE = 100

def transformed_name(key):
    return key + '_xf'

Writing loan_constants.py


In [20]:
_loan_transform_module_file = 'loan_transform.py'

In [21]:
%%writefile {_loan_transform_module_file}

import tensorflow as tf
import tensorflow_transform as tft

import loan_constants

_transformed_name = loan_constants.transformed_name
_FLOAT_FEATURE_KEYS = loan_constants.COLUMN_FLOAT
_CATEGORICAL_FEATURE_KEYS = loan_constants.COLUMN_OBJECT
_LABEL_KEY = loan_constants.LABEL_KEY
_BUCKET_SIZE = loan_constants.BUCKET_SIZE


def preprocessing_fn(inputs):
    """tf.transform's callback function for preprocessing inputs.
    Args:
    inputs: map from feature keys to raw not-yet-transformed features.
    Returns:
    Map from string feature key to transformed feature operations.
    """
    outputs = {}
    for key in _FLOAT_FEATURE_KEYS:
        # Preserve this feature as a dense float, setting nan's to the mean.
        outputs[_transformed_name(key)] = tft.scale_to_z_score(_fill_in_missing(inputs[key]))

    for key in _CATEGORICAL_FEATURE_KEYS:
        # Build a vocabulary for this feature.
        outputs[_transformed_name(key)] = tft.compute_and_apply_vocabulary(_fill_in_missing(inputs[key]), _BUCKET_SIZE)

    outputs[_transformed_name(_LABEL_KEY)] = tft.compute_and_apply_vocabulary(_fill_in_missing(inputs[_LABEL_KEY]), 7)

    return outputs


def _fill_in_missing(x):
    """Replace missing values in a SparseTensor.
    Fills in missing values of `x` with '' or 0, and converts to a dense tensor.
    Args:
    x: A `SparseTensor` of rank 2.  Its dense shape should have size at most 1
      in the second dimension.
    Returns:
    A rank 1 tensor where missing values of `x` have been filled in.
    """
    if not isinstance(x, tf.sparse.SparseTensor):
        return x
    
    default_value = '' if x.dtype == tf.string else 0
    return tf.squeeze(
      tf.sparse.to_dense(
          tf.SparseTensor(x.indices, x.values, [x.dense_shape[0], 1]),
          default_value),
      axis=1)

Writing loan_transform.py


In [22]:
transform = tfx.components.Transform(
    examples=example_gen.outputs['examples'],
    schema=schema_gen.outputs['schema'],
    module_file=os.path.abspath(_loan_transform_module_file))
context.run(transform)

INFO:absl:Generating ephemeral wheel package for '/home/jupyter/LoanDataAnalysis/loan_transform.py' (including modules: ['loan_transform', 'loan_constants']).
INFO:absl:User module package has hash fingerprint version ad43320f30444a691e39abaf2ca2f7e590617d14dd1747289c7400df28d19f98.
INFO:absl:Executing: ['/opt/conda/bin/python', '/tmp/tmpvq_s4niq/_tfx_generated_setup.py', 'bdist_wheel', '--bdist-dir', '/tmp/tmpa5vd995t', '--dist-dir', '/tmp/tmpqh9000ty']
INFO:absl:Successfully built user code wheel distribution at '/tmp/tfx-interactive-2021-08-09T00_37_07.628346-35rc043i/_wheels/tfx_user_code_Transform-0.0+ad43320f30444a691e39abaf2ca2f7e590617d14dd1747289c7400df28d19f98-py3-none-any.whl'; target user module is 'loan_transform'.
INFO:absl:Full user module path is 'loan_transform@/tmp/tfx-interactive-2021-08-09T00_37_07.628346-35rc043i/_wheels/tfx_user_code_Transform-0.0+ad43320f30444a691e39abaf2ca2f7e590617d14dd1747289c7400df28d19f98-py3-none-any.whl'
INFO:absl:Running driver for Transf

Instructions for updating:
Use ref() instead.


INFO:absl:Feature acc_now_delinq has no shape. Setting to VarLenSparseTensor.
INFO:absl:Feature acc_open_past_24mths has no shape. Setting to VarLenSparseTensor.
INFO:absl:Feature addr_state has no shape. Setting to VarLenSparseTensor.
INFO:absl:Feature all_util has no shape. Setting to VarLenSparseTensor.
INFO:absl:Feature annual_inc has no shape. Setting to VarLenSparseTensor.
INFO:absl:Feature annual_inc_joint has no shape. Setting to VarLenSparseTensor.
INFO:absl:Feature application_type has no shape. Setting to VarLenSparseTensor.
INFO:absl:Feature avg_cur_bal has no shape. Setting to VarLenSparseTensor.
INFO:absl:Feature bc_open_to_buy has no shape. Setting to VarLenSparseTensor.
INFO:absl:Feature bc_util has no shape. Setting to VarLenSparseTensor.
INFO:absl:Feature chargeoff_within_12_mths has no shape. Setting to VarLenSparseTensor.
INFO:absl:Feature collection_recovery_fee has no shape. Setting to VarLenSparseTensor.
INFO:absl:Feature collections_12_mths_ex_med has no shape. 

INFO:absl:Feature acc_now_delinq has no shape. Setting to VarLenSparseTensor.
INFO:absl:Feature acc_open_past_24mths has no shape. Setting to VarLenSparseTensor.
INFO:absl:Feature addr_state has no shape. Setting to VarLenSparseTensor.
INFO:absl:Feature all_util has no shape. Setting to VarLenSparseTensor.
INFO:absl:Feature annual_inc has no shape. Setting to VarLenSparseTensor.
INFO:absl:Feature annual_inc_joint has no shape. Setting to VarLenSparseTensor.
INFO:absl:Feature application_type has no shape. Setting to VarLenSparseTensor.
INFO:absl:Feature avg_cur_bal has no shape. Setting to VarLenSparseTensor.
INFO:absl:Feature bc_open_to_buy has no shape. Setting to VarLenSparseTensor.
INFO:absl:Feature bc_util has no shape. Setting to VarLenSparseTensor.
INFO:absl:Feature chargeoff_within_12_mths has no shape. Setting to VarLenSparseTensor.
INFO:absl:Feature collection_recovery_fee has no shape. Setting to VarLenSparseTensor.
INFO:absl:Feature collections_12_mths_ex_med has no shape. 

INFO:absl:Feature acc_now_delinq has no shape. Setting to VarLenSparseTensor.
INFO:absl:Feature acc_open_past_24mths has no shape. Setting to VarLenSparseTensor.
INFO:absl:Feature addr_state has no shape. Setting to VarLenSparseTensor.
INFO:absl:Feature all_util has no shape. Setting to VarLenSparseTensor.
INFO:absl:Feature annual_inc has no shape. Setting to VarLenSparseTensor.
INFO:absl:Feature annual_inc_joint has no shape. Setting to VarLenSparseTensor.
INFO:absl:Feature application_type has no shape. Setting to VarLenSparseTensor.
INFO:absl:Feature avg_cur_bal has no shape. Setting to VarLenSparseTensor.
INFO:absl:Feature bc_open_to_buy has no shape. Setting to VarLenSparseTensor.
INFO:absl:Feature bc_util has no shape. Setting to VarLenSparseTensor.
INFO:absl:Feature chargeoff_within_12_mths has no shape. Setting to VarLenSparseTensor.
INFO:absl:Feature collection_recovery_fee has no shape. Setting to VarLenSparseTensor.
INFO:absl:Feature collections_12_mths_ex_med has no shape. 

INFO:tensorflow:Assets written to: /tmp/tfx-interactive-2021-08-09T00_37_07.628346-35rc043i/Transform/transform_graph/5/.temp_path/tftransform_tmp/019feb03c8db4703b306bd22762103f8/assets
INFO:tensorflow:Assets written to: /tmp/tfx-interactive-2021-08-09T00_37_07.628346-35rc043i/Transform/transform_graph/5/.temp_path/tftransform_tmp/b39149ff1fa64a929d9ccc996fce3084/assets


INFO:absl:Running publisher for Transform
INFO:absl:MetadataStore with DB connection initialized


ExecutionResult(
    component_id: Transform
    execution_id: 5
    outputs:
        transform_graph: Channel(
            type_name: TransformGraph
            artifacts: [Artifact(artifact: id: 5
        type_id: 13
        uri: "/tmp/tfx-interactive-2021-08-09T00_37_07.628346-35rc043i/Transform/transform_graph/5"
        custom_properties {
          key: "name"
          value {
            string_value: "transform_graph"
          }
        }
        custom_properties {
          key: "producer_component"
          value {
            string_value: "Transform"
          }
        }
        custom_properties {
          key: "state"
          value {
            string_value: "published"
          }
        }
        custom_properties {
          key: "tfx_version"
          value {
            string_value: "1.0.0"
          }
        }
        state: LIVE
        , artifact_type: id: 13
        name: "TransformGraph"
        )]
            additional_properties: {}
            additional_custom_properties: {}
        )
        transformed_examples: Channel(
            type_name: Examples
            artifacts: [Artifact(artifact: id: 6
        type_id: 5
        uri: "/tmp/tfx-interactive-2021-08-09T00_37_07.628346-35rc043i/Transform/transformed_examples/5"
        properties {
          key: "split_names"
          value {
            string_value: "[\"train\", \"eval\"]"
          }
        }
        custom_properties {
          key: "name"
          value {
            string_value: "transformed_examples"
          }
        }
        custom_properties {
          key: "producer_component"
          value {
            string_value: "Transform"
          }
        }
        custom_properties {
          key: "state"
          value {
            string_value: "published"
          }
        }
        custom_properties {
          key: "tfx_version"
          value {
            string_value: "1.0.0"
          }
        }
        state: LIVE
        , artifact_type: id: 5
        name: "Examples"
        properties {
          key: "span"
          value: INT
        }
        properties {
          key: "split_names"
          value: STRING
        }
        properties {
          key: "version"
          value: INT
        }
        )]
            additional_properties: {}
            additional_custom_properties: {}
        )
        updated_analyzer_cache: Channel(
            type_name: TransformCache
            artifacts: [Artifact(artifact: id: 7
        type_id: 14
        uri: "/tmp/tfx-interactive-2021-08-09T00_37_07.628346-35rc043i/Transform/updated_analyzer_cache/5"
        custom_properties {
          key: "name"
          value {
            string_value: "updated_analyzer_cache"
          }
        }
        custom_properties {
          key: "producer_component"
          value {
            string_value: "Transform"
          }
        }
        custom_properties {
          key: "state"
          value {
            string_value: "published"
          }
        }
        custom_properties {
          key: "tfx_version"
          value {
            string_value: "1.0.0"
          }
        }
        state: LIVE
        , artifact_type: id: 14
        name: "TransformCache"
        )]
            additional_properties: {}
            additional_custom_properties: {}
        )
        pre_transform_schema: Channel(
            type_name: Schema
            artifacts: [Artifact(artifact: id: 8
        type_id: 9
        uri: "/tmp/tfx-interactive-2021-08-09T00_37_07.628346-35rc043i/Transform/pre_transform_schema/5"
        custom_properties {
          key: "name"
          value {
            string_value: "pre_transform_schema"
          }
        }
        custom_properties {
          key: "producer_component"
          value {
            string_value: "Transform"
          }
        }
        custom_properties {
          key: "state"
          value {
            string_value: "published"
   

In [23]:
transform.outputs

{'transform_graph': Channel(
     type_name: TransformGraph
     artifacts: [Artifact(artifact: id: 5
 type_id: 13
 uri: "/tmp/tfx-interactive-2021-08-09T00_37_07.628346-35rc043i/Transform/transform_graph/5"
 custom_properties {
   key: "name"
   value {
     string_value: "transform_graph"
   }
 }
 custom_properties {
   key: "producer_component"
   value {
     string_value: "Transform"
   }
 }
 custom_properties {
   key: "state"
   value {
     string_value: "published"
   }
 }
 custom_properties {
   key: "tfx_version"
   value {
     string_value: "1.0.0"
   }
 }
 state: LIVE
 , artifact_type: id: 13
 name: "TransformGraph"
 )]
     additional_properties: {}
     additional_custom_properties: {}
 ),
 'transformed_examples': Channel(
     type_name: Examples
     artifacts: [Artifact(artifact: id: 6
 type_id: 5
 uri: "/tmp/tfx-interactive-2021-08-09T00_37_07.628346-35rc043i/Transform/transformed_examples/5"
 properties {
   key: "split_names"
   value {
     string_value: "[\"trai

In [24]:
train_uri = transform.outputs['transform_graph'].get()[0].uri
os.listdir(train_uri)

['transform_fn', 'metadata', 'transformed_metadata']

In [25]:
# Get the URI of the output artifact representing the transformed examples, which is a directory
train_uri = os.path.join(transform.outputs['transformed_examples'].get()[0].uri, 'Split-train')

# Get the list of files in this directory (all compressed TFRecord files)
tfrecord_filenames = [os.path.join(train_uri, name)
                      for name in os.listdir(train_uri)]

# Create a `TFRecordDataset` to read these files
dataset = tf.data.TFRecordDataset(tfrecord_filenames, compression_type="GZIP")

# Iterate over the first 3 records and decode them.
# for tfrecord in dataset.take(10):
#     serialized_example = tfrecord.numpy()
#     example = tf.train.Example()
#     example.ParseFromString(serialized_example)
#     pp.pprint(example)

In [26]:
_loan_trainer_module_file = 'loan_trainer.py'

In [27]:
%%writefile {_loan_trainer_module_file}

from typing import List, Text

import os
import absl
import datetime
import tensorflow as tf
import tensorflow_transform as tft

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers

from tfx import v1 as tfx
from tfx_bsl.public import tfxio

import loan_constants

_FLOAT_FEATURE_KEYS = loan_constants.COLUMN_FLOAT
_CATEGORICAL_FEATURE_KEYS = loan_constants.COLUMN_OBJECT
_transformed_name = loan_constants.transformed_name
_LABEL_KEY = loan_constants.LABEL_KEY
_BUCKET_SIZE = loan_constants.BUCKET_SIZE


def _transformed_names(keys):
    return [_transformed_name(key) for key in keys]


def _get_serve_tf_examples_fn(model, tf_transform_output):
    """Returns a function that parses a serialized tf.Example and applies TFT."""

    model.tft_layer = tf_transform_output.transform_features_layer()

    @tf.function
    def serve_tf_examples_fn(serialized_tf_examples):
        """Returns the output to be used in the serving signature."""
        feature_spec = tf_transform_output.raw_feature_spec()
        feature_spec.pop(_LABEL_KEY)
        parsed_features = tf.io.parse_example(serialized_tf_examples, feature_spec)
        transformed_features = model.tft_layer(parsed_features)
        return model(transformed_features)

    return serve_tf_examples_fn


def _input_fn(file_pattern: List[Text],
              data_accessor: tfx.components.DataAccessor,
              tf_transform_output: tft.TFTransformOutput,
              batch_size: int = 200) -> tf.data.Dataset:
    """Generates features and label for tuning/training.

    Args:
    file_pattern: List of paths or patterns of input tfrecord files.
    data_accessor: DataAccessor for converting input to RecordBatch.
    tf_transform_output: A TFTransformOutput.
    batch_size: representing the number of consecutive elements of returned
      dataset to combine in a single batch

    Returns:
    A dataset that contains (features, indices) tuple where features is a
      dictionary of Tensors, and indices is a single Tensor of label indices.
    """
    return data_accessor.tf_dataset_factory(
      file_pattern,
      tfxio.TensorFlowDatasetOptions(
          batch_size=batch_size, label_key=_transformed_name(_LABEL_KEY)),
      tf_transform_output.transformed_metadata.schema)


def _build_keras_model() -> tf.keras.Model:
    """Creates a DNN Keras model for classifying Loan data.

    Returns:
    A keras Model.
    """
    real_valued_columns = [
      tf.feature_column.numeric_column(key, shape=())
      for key in _transformed_names(_FLOAT_FEATURE_KEYS)
    ]
    categorical_columns = [
      tf.feature_column.categorical_column_with_identity(
          key, num_buckets=_BUCKET_SIZE, default_value=0)
      for key in _transformed_names(_CATEGORICAL_FEATURE_KEYS)
    ]


    indicator_column = [
      tf.feature_column.indicator_column(categorical_column)
      for categorical_column in categorical_columns
    ]
    

    model = _wide_and_deep_classifier(
      wide_columns=indicator_column,
      deep_columns=real_valued_columns)
    return model


def _wide_and_deep_classifier(wide_columns, deep_columns):
    """Build a simple keras wide and deep model.

    Args:
    wide_columns: Feature columns wrapped in indicator_column for wide (linear)
      part of the model.
    deep_columns: Feature columns for deep part of the model.
    dnn_hidden_units: [int], the layer sizes of the hidden DNN.

    Returns:
    A Wide and Deep Keras model
    """

    # Keras needs the feature definitions at compile time.
    input_layers = {
      colname: tf.keras.layers.Input(name=colname, shape=(), dtype=tf.float32)
      for colname in _transformed_names(_FLOAT_FEATURE_KEYS)
    }
    input_layers.update({
      colname: tf.keras.layers.Input(name=colname, shape=(), dtype='int32')
      for colname in _transformed_names(_CATEGORICAL_FEATURE_KEYS)
    })

    deep = tf.keras.layers.DenseFeatures(deep_columns)(input_layers)
    deep = tf.keras.layers.Dense(128, activation="relu")(deep)
    deep = tf.keras.layers.Dense(64, activation="relu")(deep)
    wide = tf.keras.layers.DenseFeatures(wide_columns)(input_layers)

    output = tf.keras.layers.Dense(7, activation="softmax")(tf.keras.layers.concatenate([deep, wide]))

    model = tf.keras.Model(input_layers, output)

    model.compile(loss=tf.keras.losses.SparseCategoricalCrossentropy(), optimizer=tf.keras.optimizers.Adam(), metrics=['accuracy'])

    return model


# TFX Trainer will call this function.
def run_fn(fn_args: tfx.components.FnArgs):
    """Train the model based on given args.

    Args:
    fn_args: Holds args used to train the model as name/value pairs.
    """

    tf_transform_output = tft.TFTransformOutput(fn_args.transform_output)

    train_dataset = _input_fn(fn_args.train_files, fn_args.data_accessor, 
                            tf_transform_output, 32)
    eval_dataset = _input_fn(fn_args.eval_files, fn_args.data_accessor, 
                           tf_transform_output, 32)

    model = _build_keras_model()

    tensorboard_callback = tf.keras.callbacks.TensorBoard(
      log_dir=fn_args.model_run_dir, update_freq='batch')
    model.fit(
      train_dataset,
      steps_per_epoch=fn_args.train_steps,
      validation_data=eval_dataset,
      validation_steps=fn_args.eval_steps,
      callbacks=[tensorboard_callback])

    signatures = {
      'serving_default':
          _get_serve_tf_examples_fn(model,
                                    tf_transform_output).get_concrete_function(
                                        tf.TensorSpec(
                                            shape=[None],
                                            dtype=tf.string,
                                            name='examples')),
    }
    model.save(fn_args.serving_model_dir, save_format='tf', signatures=signatures)

Writing loan_trainer.py


In [28]:
trainer = tfx.components.Trainer(
    module_file=os.path.abspath(_loan_trainer_module_file),
    examples=transform.outputs['transformed_examples'],
    transform_graph=transform.outputs['transform_graph'],
    schema=schema_gen.outputs['schema'],
    train_args=tfx.proto.TrainArgs(num_steps=10000),
    eval_args=tfx.proto.EvalArgs(num_steps=5000))
context.run(trainer)

INFO:absl:Generating ephemeral wheel package for '/home/jupyter/LoanDataAnalysis/loan_trainer.py' (including modules: ['loan_transform', 'loan_trainer', 'loan_constants']).
INFO:absl:User module package has hash fingerprint version a7caf91509fe25897c526fd4e16fd54e9e8257f256b4a80068f4812bb943defb.
INFO:absl:Executing: ['/opt/conda/bin/python', '/tmp/tmpeqe1pcu_/_tfx_generated_setup.py', 'bdist_wheel', '--bdist-dir', '/tmp/tmpoxb3s5bk', '--dist-dir', '/tmp/tmplcf08z9k']
INFO:absl:Successfully built user code wheel distribution at '/tmp/tfx-interactive-2021-08-09T00_37_07.628346-35rc043i/_wheels/tfx_user_code_Trainer-0.0+a7caf91509fe25897c526fd4e16fd54e9e8257f256b4a80068f4812bb943defb-py3-none-any.whl'; target user module is 'loan_trainer'.
INFO:absl:Full user module path is 'loan_trainer@/tmp/tfx-interactive-2021-08-09T00_37_07.628346-35rc043i/_wheels/tfx_user_code_Trainer-0.0+a7caf91509fe25897c526fd4e16fd54e9e8257f256b4a80068f4812bb943defb-py3-none-any.whl'
INFO:absl:Running driver for 

10000/10000 [==============================] - ETA: 9:29:34 - loss: 1.9443 - accuracy: 0.093 - ETA: 42:05 - loss: 1.9775 - accuracy: 0.0938  - ETA: 26:00 - loss: 1.9542 - accuracy: 0.156 - ETA: 10:10 - loss: 1.9232 - accuracy: 0.147 - ETA: 9:13 - loss: 1.9012 - accuracy: 0.156 - ETA: 6:57 - loss: 1.8493 - accuracy: 0.21 - ETA: 5:47 - loss: 1.8077 - accuracy: 0.24 - ETA: 5:53 - loss: 1.7863 - accuracy: 0.26 - ETA: 5:15 - loss: 1.7490 - accuracy: 0.27 - ETA: 4:47 - loss: 1.7279 - accuracy: 0.30 - ETA: 4:27 - loss: 1.6965 - accuracy: 0.31 - ETA: 4:12 - loss: 1.6857 - accuracy: 0.30 - ETA: 3:59 - loss: 1.6653 - accuracy: 0.31 - ETA: 3:49 - loss: 1.6398 - accuracy: 0.32 - ETA: 3:52 - loss: 1.6309 - accuracy: 0.32 - ETA: 3:58 - loss: 1.6247 - accuracy: 0.32 - ETA: 3:50 - loss: 1.5925 - accuracy: 0.34 - ETA: 3:43 - loss: 1.5754 - accuracy: 0.35 - ETA: 3:37 - loss: 1.5485 - accuracy: 0.36 - ETA: 3:32 - loss: 1.5325 - accuracy: 0.36 - ETA: 3:34 - loss: 1.5249 - accuracy: 0.37 - ETA: 3:34 - loss

INFO:absl:Training complete. Model written to /tmp/tfx-interactive-2021-08-09T00_37_07.628346-35rc043i/Trainer/model/6/Format-Serving. ModelRun written to /tmp/tfx-interactive-2021-08-09T00_37_07.628346-35rc043i/Trainer/model_run/6
INFO:absl:Running publisher for Trainer
INFO:absl:MetadataStore with DB connection initialized


ExecutionResult(
    component_id: Trainer
    execution_id: 6
    outputs:
        model: Channel(
            type_name: Model
            artifacts: [Artifact(artifact: id: 13
        type_id: 16
        uri: "/tmp/tfx-interactive-2021-08-09T00_37_07.628346-35rc043i/Trainer/model/6"
        custom_properties {
          key: "name"
          value {
            string_value: "model"
          }
        }
        custom_properties {
          key: "producer_component"
          value {
            string_value: "Trainer"
          }
        }
        custom_properties {
          key: "state"
          value {
            string_value: "published"
          }
        }
        custom_properties {
          key: "tfx_version"
          value {
            string_value: "1.0.0"
          }
        }
        state: LIVE
        , artifact_type: id: 16
        name: "Model"
        )]
            additional_properties: {}
            additional_custom_properties: {}
        )
        model_run: Channel(
            type_name: ModelRun
            artifacts: [Artifact(artifact: id: 14
        type_id: 17
        uri: "/tmp/tfx-interactive-2021-08-09T00_37_07.628346-35rc043i/Trainer/model_run/6"
        custom_properties {
          key: "name"
          value {
            string_value: "model_run"
          }
        }
        custom_properties {
          key: "producer_component"
          value {
            string_value: "Trainer"
          }
        }
        custom_properties {
          key: "state"
          value {
            string_value: "published"
          }
        }
        custom_properties {
          key: "tfx_version"
          value {
            string_value: "1.0.0"
          }
        }
        state: LIVE
        , artifact_type: id: 17
        name: "ModelRun"
        )]
            additional_properties: {}
            additional_custom_properties: {}
        ))

In [29]:
model_artifact_dir = trainer.outputs['model'].get()[0].uri
pp.pprint(os.listdir(model_artifact_dir))
model_dir = os.path.join(model_artifact_dir, 'Format-Serving')
pp.pprint(os.listdir(model_dir))

['Format-Serving']
['assets', 'variables', 'saved_model.pb', 'keras_metadata.pb']


In [30]:
model_run_artifact_dir = trainer.outputs['model_run'].get()[0].uri

%load_ext tensorboard
%tensorboard --logdir {model_run_artifact_dir}

In [35]:
eval_config = tfma.EvalConfig(
    model_specs=[
        # This assumes a serving model with signature 'serving_default'. If
        # using estimator based EvalSavedModel, add signature_name: 'eval' and 
        # remove the label_key.
        tfma.ModelSpec(label_key='grade_xf')
    ],
    metrics_specs=[
        tfma.MetricsSpec(
            # The metrics added here are in addition to those saved with the
            # model (assuming either a keras model or EvalSavedModel is used).
            # Any metrics added into the saved model (for example using
            # model.compile(..., metrics=[...]), etc) will be computed
            # automatically.
            # To add validation thresholds for metrics saved with the model,
            # add them keyed by metric name to the thresholds map.
            metrics=[
                tfma.MetricConfig(class_name='ExampleCount'),
                tfma.MetricConfig(class_name='Accuracy',
                  threshold=tfma.MetricThreshold(
                      value_threshold=tfma.GenericValueThreshold(
                          lower_bound={'value': 0.9}),
                      # Change threshold will be ignored if there is no
                      # baseline model resolved from MLMD (first run).
                      change_threshold=tfma.GenericChangeThreshold(
                          direction=tfma.MetricDirection.HIGHER_IS_BETTER,
                          absolute={'value': -1e-10})))
            ]
        )
    ],
    slicing_specs=[
        # An empty slice spec means the overall slice, i.e. the whole dataset.
        tfma.SlicingSpec(),
        # Data can be sliced along a feature column. In this case, data is
        # sliced along feature column trip_start_hour.
        tfma.SlicingSpec(feature_keys=[''])
    ]
)

In [36]:
# Use TFMA to compute a evaluation statistics over features of a model and
# validate them against a baseline.

# The model resolver is only required if performing model validation in addition
# to evaluation. In this case we validate against the latest blessed model. If
# no model has been blessed before (as in this case) the evaluator will make our
# candidate the first blessed model.
model_resolver = tfx.dsl.Resolver(
      strategy_class=tfx.dsl.experimental.LatestBlessedModelStrategy,
      model=tfx.dsl.Channel(type=tfx.types.standard_artifacts.Model),
      model_blessing=tfx.dsl.Channel(
          type=tfx.types.standard_artifacts.ModelBlessing)).with_id(
              'latest_blessed_model_resolver')
context.run(model_resolver)

evaluator = tfx.components.Evaluator(
    examples=example_gen.outputs['examples'],
    model=trainer.outputs['model'],
    baseline_model=model_resolver.outputs['model'],
    eval_config=eval_config)
context.run(evaluator)

INFO:absl:Running driver for latest_blessed_model_resolver
INFO:absl:MetadataStore with DB connection initialized
INFO:absl:Running publisher for latest_blessed_model_resolver
INFO:absl:MetadataStore with DB connection initialized
INFO:absl:Running driver for Evaluator
INFO:absl:MetadataStore with DB connection initialized
INFO:absl:Running executor for Evaluator
INFO:absl:Nonempty beam arg extra_packages already includes dependency
INFO:absl:udf_utils.get_fn {'eval_config': '{\n  "metrics_specs": [\n    {\n      "metrics": [\n        {\n          "class_name": "ExampleCount"\n        },\n        {\n          "class_name": "Accuracy",\n          "threshold": {\n            "change_threshold": {\n              "absolute": -1e-10,\n              "direction": "HIGHER_IS_BETTER"\n            },\n            "value_threshold": {\n              "lower_bound": 0.9\n            }\n          }\n        }\n      ]\n    }\n  ],\n  "model_specs": [\n    {\n      "label_key": "grade_xf"\n    }\n  ]


Two checkpoint references resolved to different objects (<tensorflow.python.keras.saving.saved_model.load.TensorFlowTransform>TransformFeaturesLayer object at 0x7fd9babe18d0> and <tensorflow.python.keras.engine.input_layer.InputLayer object at 0x7fd98e5bbd90>).


INFO:absl:The 'example_splits' parameter is not set, using 'eval' split.
INFO:absl:Evaluating model.
INFO:absl:udf_utils.get_fn {'eval_config': '{\n  "metrics_specs": [\n    {\n      "metrics": [\n        {\n          "class_name": "ExampleCount"\n        },\n        {\n          "class_name": "Accuracy",\n          "threshold": {\n            "change_threshold": {\n              "absolute": -1e-10,\n              "direction": "HIGHER_IS_BETTER"\n            },\n            "value_threshold": {\n              "lower_bound": 0.9\n            }\n          }\n        }\n      ]\n    }\n  ],\n  "model_specs": [\n    {\n      "label_key": "grade_xf"\n    }\n  ],\n  "slicing_specs": [\n    {},\n    {\n      "feature_keys": [\n        ""\n      ]\n    }\n  ]\n}', 'feature_slicing_spec': None, 'fairness_indicator_thresholds': None, 'example_splits': 'null', 'module_file': None, 'module_path': None} 'custom_extractors'
INFO:absl:Request was made to ignore the baseline ModelSpec and any change t


Two checkpoint references resolved to different objects (<tensorflow.python.keras.saving.saved_model.load.TensorFlowTransform>TransformFeaturesLayer object at 0x7fd957ff1f50> and <tensorflow.python.keras.engine.input_layer.InputLayer object at 0x7fd98e5bb850>).



Two checkpoint references resolved to different objects (<tensorflow.python.keras.saving.saved_model.load.TensorFlowTransform>TransformFeaturesLayer object at 0x7fd9550dead0> and <tensorflow.python.keras.engine.input_layer.InputLayer object at 0x7fd9551f4310>).


ValueError: no value provided for label: model_name=, output_name=, sub_key=None, aggregation_type=None, StandardMetricInputs=<tensorflow_model_analysis.metrics.metric_types.StandardMetricInputs object at 0x7fd9550909d0>

This may be caused by a configuration error (i.e. label, and/or prediction keys were not specified) or an error in the pipeline. [while running 'ExtractEvaluateAndWriteResults/ExtractAndEvaluate/EvaluateMetricsAndPlots/ComputeMetricsAndPlots()/ComputePerSlice/ComputeUnsampledMetrics/CombinePerSliceKey/WindowIntoDiscarding']

In [ ]:
evaluator.outputs

In [ ]:
context.show(evaluator.outputs['evaluation'])

In [ ]:
import tensorflow_model_analysis as tfma

# Get the TFMA output result path and load the result.
PATH_TO_RESULT = evaluator.outputs['evaluation'].get()[0].uri
tfma_result = tfma.load_eval_result(PATH_TO_RESULT)

# Show data sliced along feature column loan_amnt.
tfma.view.render_slicing_metrics(
    tfma_result, slicing_column='loan_amnt')

In [ ]:
blessing_uri = evaluator.outputs['blessing'].get()[0].uri
!ls -l {blessing_uri}

In [ ]:
PATH_TO_RESULT = evaluator.outputs['evaluation'].get()[0].uri
print(tfma.load_validation_result(PATH_TO_RESULT))

In [ ]:
pusher = tfx.components.Pusher(
    model=trainer.outputs['model'],
    model_blessing=evaluator.outputs['blessing'],
    push_destination=tfx.proto.PushDestination(
        filesystem=tfx.proto.PushDestination.Filesystem(
            base_directory=_serving_model_dir)))
context.run(pusher)

In [ ]:
pusher.outputs

In [ ]:
push_uri = pusher.outputs['pushed_model'].get()[0].uri
model = tf.saved_model.load(push_uri)

for item in model.signatures.items():
    pp.pprint(item)